# Exercise: Customer Banking Data Integration & Imputation


Notebook ini merupakan bagian dari latihan eksplorasi data pelanggan bank. Fokus utama adalah penggabungan beberapa tabel data dan penanganan nilai kosong (missing value) pada informasi demografi dan transaksi pelanggan.



### 📁 Dataset Sumber

- `customer_demography`
- `savingmaster`
- `transaction`

Ketiga dataset ini akan digabungkan berdasarkan kolom `customer_id`.


In [120]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [128]:
df_demography = pd.read_excel('C:/Users/ahmad/OneDrive/Documents/Bootcamp/Modul 2/Python for Data Analyst/Session 13/Practice/Case Study/dataset/Demoghraphy.xlsx')
df_saving = pd.read_excel('C:/Users/ahmad/OneDrive/Documents/Bootcamp/Modul 2/Python for Data Analyst/Session 13/Practice/Case Study/dataset/Savingmaster.xlsx')
df_trx = pd.read_csv('C:/Users/ahmad/OneDrive/Documents/Bootcamp/Modul 2/Python for Data Analyst/Session 13/Practice/Case Study/dataset/trx.csv')

## Bagian 1: Data Join dan Validasi

### 1. Lakukan join antara `customer_demography` dan `savingmaster` berdasarkan kolom `customer_id`.

In [148]:
df_join1 = pd.merge(df_demography, df_saving, on='customer_id', how='left')
df_join1.head()

,customer_id,account_id,umur,region,jenis_kelamin,status_nikah,jenis_pekerjaan,jenis_merchant,has_sms_fin,has_internet_fin,has_tabungan,has_giro,has_deposito,has_loan,umur_rekening
0,c0001,614061949546,30.0,K,M,B,Others,RITEL,0,1,1,0,0,0,59
1,c0002,717073892379,28.0,G,M,B,Wiraswasta,RITEL,0,0,1,0,0,1,49
2,c0003,930922328467,38.0,R,NaN,K,Wiraswasta,RITEL,1,1,1,1,0,1,54
3,c0004,539173601521,36.0,N,M,K,Pedagang,RITEL,0,1,1,0,0,0,132
4,c0005,504741789560,53.0,G,M,K,Wiraswasta,RITEL,1,0,1,0,0,1,117


### 2. Gabungkan hasil dari langkah pertama dengan `transaction` menggunakan kolom `customer_id`.

In [130]:
df_gabung = pd.merge(df_join1, df_trx, on='account_id', how='left')
df_gabung.head()

,customer_id,account_id,umur,region,jenis_kelamin,status_nikah,jenis_pekerjaan,jenis_merchant,has_sms_fin,has_internet_fin,...,amt_debit_m3,amt_debit_m4,amt_debit_m5,amt_debit_m6,amt_credit_m1,amt_credit_m2,amt_credit_m3,amt_credit_m4,amt_credit_m5,amt_credit_m6
0,c0001,614061949546,30.0,K,M,B,Others,RITEL,0,1,...,0.0,0.0,0.0,0.0,100000.0,0.0,0.0,0.0,0.0,0.0
1,c0001,614061949546,30.0,K,M,B,Others,RITEL,0,1,...,0.0,0.0,0.0,0.0,100000.0,0.0,0.0,0.0,0.0,0.0
2,c0002,717073892379,28.0,G,M,B,Wiraswasta,RITEL,0,0,...,0.0,0.0,0.0,0.0,40690328.0,0.0,0.0,0.0,0.0,0.0
3,c0002,717073892379,28.0,G,M,B,Wiraswasta,RITEL,0,0,...,0.0,0.0,0.0,0.0,40690328.0,0.0,0.0,0.0,0.0,0.0
4,c0003,930922328467,38.0,R,NaN,K,Wiraswasta,RITEL,1,1,...,0.0,0.0,0.0,0.0,9785232.0,11205062.0,0.0,0.0,0.0,0.0


### 3. Pastikan hasil akhir memiliki **8.397 baris** sesuai jumlah pelanggan pada data demografi.

In [149]:
df_gabung.shape

(11397, 65)

## Bagian 2: Imputasi Missing Values

### 4. Lakukan imputasi pada kolom-kolom berikut yang memiliki nilai kosong:

- `age`  
- `gender`  
- `marital_status`  
- `occupation`  
- `freq_debit`  
- `freq_credit`

In [138]:
# Buat list kolom per bulan
debit = [col for col in df_gabung.columns if 'freq_debit_m' in col]
credit = [col for col in df_gabung.columns if 'freq_credit_m' in col]

In [137]:
# Agregasi jumlah total frekuensi 6 bulan
df_gabung['freq_debit'] = df_gabung[debit].sum(axis=1)
df_gabung['freq_credit'] = df_gabung[credit].sum(axis=1)

In [140]:
kolom_untuk_imputasi = ['umur', 'jenis_kelamin', 'status_nikah', 'jenis_pekerjaan', 'freq_debit', 'freq_credit']
df_gabung[kolom_untuk_imputasi].isnull().sum()

umur               2256
jenis_kelamin       573
status_nikah       1147
jenis_pekerjaan    1163
freq_debit            0
freq_credit           0
dtype: int64

### 5. Bebas memilih teknik imputasi (mean/median/mode/domain knowledge).

In [125]:
median_age = df_gabung['umur'].median()
df_gabung['umur'].fillna(median_age, inplace=True)
df_gabung

,customer_id,account_id,umur,region,jenis_kelamin,status_nikah,jenis_pekerjaan,jenis_merchant,has_sms_fin,has_internet_fin,...,amt_debit_m3,amt_debit_m4,amt_debit_m5,amt_debit_m6,amt_credit_m1,amt_credit_m2,amt_credit_m3,amt_credit_m4,amt_credit_m5,amt_credit_m6
0,c0001,614061949546,30.0,K,M,B,Others,RITEL,0,1,...,0.0,0.0,0.0,0.0,1.000000e+05,0.0,0.0,0.0,0.0,0.0
1,c0001,614061949546,30.0,K,M,B,Others,RITEL,0,1,...,0.0,0.0,0.0,0.0,1.000000e+05,0.0,0.0,0.0,0.0,0.0
2,c0002,717073892379,28.0,G,M,B,Wiraswasta,RITEL,0,0,...,0.0,0.0,0.0,0.0,4.069033e+07,0.0,0.0,0.0,0.0,0.0
3,c0002,717073892379,28.0,G,M,B,Wiraswasta,RITEL,0,0,...,0.0,0.0,0.0,0.0,4.069033e+07,0.0,0.0,0.0,0.0,0.0
4,c0003,930922328467,38.0,R,M,K,Wiraswasta,RITEL,1,1,...,0.0,0.0,0.0,0.0,9.785232e+06,11205062.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11392,c8393,154397542586,41.0,Q,M,K,Wiraswasta,RITEL,1,1,...,0.0,0.0,0.0,0.0,5.429349e+07,43818192.0,0.0,0.0,0.0,0.0
11393,c8394,758666458531,46.0,F,F,K,Pegawai Swasta,RITEL,1,1,...,0.0,0.0,0.0,0.0,7.028183e+08,0.0,0.0,0.0,0.0,0.0
11394,c8395,322213539385,51.0,H,F,B,Others,RITEL,1,1,...,0.0,0.0,0.0,0.0,9.575165e+07,0.0,0.0,0.0,0.0,0.0
11395,c8396,683754973926,28.0,B,F,B,Others,PARTNERSHIP,0,1,...,373684196.4,542236813.0,422156420.0,254693998.0,4.602352e+08,567509464.0,330671761.0,614908321.0,416581768.0,238476915.0


In [126]:
mode_gender = df_gabung['jenis_kelamin'].mode()[0]
df_gabung['jenis_kelamin'].fillna(mode_gender, inplace=True)
df_gabung

,customer_id,account_id,umur,region,jenis_kelamin,status_nikah,jenis_pekerjaan,jenis_merchant,has_sms_fin,has_internet_fin,...,amt_debit_m3,amt_debit_m4,amt_debit_m5,amt_debit_m6,amt_credit_m1,amt_credit_m2,amt_credit_m3,amt_credit_m4,amt_credit_m5,amt_credit_m6
0,c0001,614061949546,30.0,K,M,B,Others,RITEL,0,1,...,0.0,0.0,0.0,0.0,1.000000e+05,0.0,0.0,0.0,0.0,0.0
1,c0001,614061949546,30.0,K,M,B,Others,RITEL,0,1,...,0.0,0.0,0.0,0.0,1.000000e+05,0.0,0.0,0.0,0.0,0.0
2,c0002,717073892379,28.0,G,M,B,Wiraswasta,RITEL,0,0,...,0.0,0.0,0.0,0.0,4.069033e+07,0.0,0.0,0.0,0.0,0.0
3,c0002,717073892379,28.0,G,M,B,Wiraswasta,RITEL,0,0,...,0.0,0.0,0.0,0.0,4.069033e+07,0.0,0.0,0.0,0.0,0.0
4,c0003,930922328467,38.0,R,M,K,Wiraswasta,RITEL,1,1,...,0.0,0.0,0.0,0.0,9.785232e+06,11205062.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11392,c8393,154397542586,41.0,Q,M,K,Wiraswasta,RITEL,1,1,...,0.0,0.0,0.0,0.0,5.429349e+07,43818192.0,0.0,0.0,0.0,0.0
11393,c8394,758666458531,46.0,F,F,K,Pegawai Swasta,RITEL,1,1,...,0.0,0.0,0.0,0.0,7.028183e+08,0.0,0.0,0.0,0.0,0.0
11394,c8395,322213539385,51.0,H,F,B,Others,RITEL,1,1,...,0.0,0.0,0.0,0.0,9.575165e+07,0.0,0.0,0.0,0.0,0.0
11395,c8396,683754973926,28.0,B,F,B,Others,PARTNERSHIP,0,1,...,373684196.4,542236813.0,422156420.0,254693998.0,4.602352e+08,567509464.0,330671761.0,614908321.0,416581768.0,238476915.0



Pastikan semua proses berjalan dengan baik dan data telah siap untuk tahap eksplorasi dan analisis lanjutan, seperti segmentasi pelanggan atau klasifikasi risiko.
